In [1]:
import numpy as np
# import cv2
import time
import os
import os.path
import glob
import threading
import sys
import smtplib
%matplotlib auto

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker import Tracker
from Tracker.Tracker import Tracker

Using matplotlib backend: TkAgg


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# RE-ANALYZE VIDEOS 

# define thread of getting user input to close down
def get_input():
    global flag
    print('')
    keystrk=input('\n\nPress a key to exit: ')
    # thread doesn't continue until key is pressed
    flag=False
    print('\nYou pressed: ', keystrk, ' -- PROGRAM EXITING\n')
    print('')

    return flag;

def trackvid(filename, min_osize, thresh_val):
    # contours tracking etc.
    video = Tracker(filename, min_object_size= min_osize)
    try:
        if video.file_exists == False:

            video.threshold_val = thresh_val
            video.load_video()
            video.compute_background()          # form background image
            video.remove_background()           # remove background
            video.threshold()                   # threshold to segment features
            video.find_distance()               # remove dist before closing on clear bridge
            video.morpho_closing()
            video.find_objects()
            video.draw_contours()
            video.save_JSON()
            video.associate_contours(max_covariance=10,
                             max_velocity=100,
                             n_covariances_to_reject=20, 
                             max_tracked_objects=100,
                             kalman_state_cov=1,
                             kalman_init_cov=0.2,
                             kalman_measurement_cov=1)
            video.save_association_JSON()

#             print('--- Contours and tracks saved')
            print(filename, ' --- tracked ') #, filename)
        else:
            print('--- Tracked files already exist')
            
    except:
        print(filename, ' --- error! ') #, filename)
        

# i created a gmail account just for this purpose
username = 'codefromglenna'
password = 'imacomputer'
fromaddr = 'codefromglenna@gmail.com'
toaddr   = '6106392662@vtext.com'

def sendtxt(msg):
    server = smtplib.SMTP('smtp.gmail.com:587');
#     server.connect();
    server.starttls();
    server.login(username,password);
    server.sendmail(fromaddr, toaddr, msg) # -- ' + time.ctime() )
    server.quit()
    

        
        


In [5]:
# vids = glob.glob('/media/gravishlab/Seagate Expansion Drive/AntTrack/Tunnel_20180502-03/*/*162767*.mp4')
vids = glob.glob('/media/gravishlab/SeagateExpansionDrive/AntTrack/*/*/*162767*.mp4')
vids = sorted(vids)[0:3]
nvids = len(vids)
n_vidsanalyzed = 0
        
# set up user input to close down program at any time
i=threading.Thread(target=get_input, args = ())#         else:
#             print(' --- files already exist')
i.start()

if nvids > 0:

    sep = 200 # separation from beginning and end of video --> frames that will analyze
    buffer = 70 # how many pixels near walls to ignore (don't count ants walking on walls)

    for vname in vids:
        
        if not i.isAlive():
            print('User input has ended. Closing...')
            time.sleep(20)
            break
            
        if n_vidsanalyzed == 0:
            print('Analyzing ' + str(nvids) + ' videos! -- ' + time.strftime('%I:%M:%S %p') + '\n')
            msg = 'Analyzing ' + str(nvids) + ' videos!' # ' -- ' + time.strftime('%I:%M %p')
            sendtxt(msg)

            
#         print(vname)
        
        # Delete contours and association files
        vname_base = vname.split('.')[:-1][0]
        cname = vname_base + '_contours.txt'
        tname = vname_base + '_tracks.txt'
        if os.path.exists(cname):
            os.system('gvfs-trash \'' + cname + '\'')
        if os.path.exists(tname):
            os.system('gvfs-trash \'' + tname + '\'')

        trackvid(vname, 500, 0.2) # filename, min object size, threshold value
        n_vidsanalyzed = n_vidsanalyzed + 1

        if n_vidsanalyzed > 0:
            if n_vidsanalyzed % 50 == 0:
                msg = 'Finished analyzing ' + str(n_vidsanalyzed) + ' videos!'
                sendtxt(msg)
                print('\nAnalyzed ' + str(n_vidsanalyzed) + ' videos -- ' + time.strftime('%I:%M:%S %p') + '\n')

        if n_vidsanalyzed == nvids:
            msg = 'Finished analyzing ' + str(n_vidsanalyzed) + ' videos!' # ' -- ' + time.strftime('%I:%M %p')
            sendtxt(msg)
            print('\nFinished analyzing ' + str(n_vidsanalyzed) + ' videos! -- ' + time.strftime('%I:%M:%S %p') + '\n')
        
            
        
print('all done!')

Analyzing 3 videos! -- 02:39:13 PM


/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080112_16276736-0000.mp4  --- tracked 
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080315_16276736-0000.mp4  --- tracked 
/media/gravishlab/SeagateExpansionDrive/AntTrack/Tunnel_20180313-14/0mm/20180313_080418_16276736-0000.mp4  --- tracked 

Finished analyzing 3 videos! -- 02:40:06 PM

all done!


In [13]:
%qtconsole

In [22]:
# find list of all videos
vids = glob.glob('/media/gravishlab/Seagate Expansion Drive/AntTrack/**/*12-0000.avi')
print('Total Number of Videos: ',len(vids))

Total Number of Videos:  343


In [23]:
for kk, vname in enumerate(sorted(vids[0:2])): #[129:130]: # total 131
    print(kk, ' --- ', vname)
    # Load in images to memory during construction
    print(vname)
    video = Tracker.Tracker(vname,min_object_size=40) # ROI= (30, 30, 550, 1150) , verbose='False'
    print(video.file_exists)
#     # video.bkg_sep = 20
#     video.threshold_val = 0.7

#     video.load_video()
#     video.compute_background()          # form background image
#     video.remove_background()           # remove background
#     video.threshold()                   # threshold to segment features
#     video.morpho_closing()
#     video.find_objects()
#     video.draw_contours()
#     video.save_JSON()
    print('---Finished Tracking')

0  ---  /media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41/20171117_163816_16276712-0000.avi
/media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41/20171117_163816_16276712-0000.avi
True
---Finished Tracking
1  ---  /media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41/20171117_175031_16276712-0000.avi
/media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41/20171117_175031_16276712-0000.avi
True
---Finished Tracking


In [14]:
for root, dirs, files in os.walk('/media/gravishlab/Seagate Expansion Drive/AntTrack/'):
    print(root)

/media/gravishlab/Seagate Expansion Drive/AntTrack/
/media/gravishlab/Seagate Expansion Drive/AntTrack/Extrinsic
/media/gravishlab/Seagate Expansion Drive/AntTrack/Extrinsic/2017_11_17
/media/gravishlab/Seagate Expansion Drive/AntTrack/Extrinsic/2017_12_05
/media/gravishlab/Seagate Expansion Drive/AntTrack/Rugosity_1_41
/media/gravishlab/Seagate Expansion Drive/AntTrack/SetupPics
/media/gravishlab/Seagate Expansion Drive/AntTrack/Smooth
